In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import transformers
from transformers import AutoTokenizer, AutoModel
from multimolecule import RnaTokenizer, RnaFmModel
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import copy
import gc
import os
import shutil
import time

In [2]:

from peft import get_peft_model, LoraConfig, TaskType, PeftModel

In [ ]:

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Apple Silicon GPU)")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA")
else:
    device = torch.device("cpu")
    print("Using CPU")

In [ ]:
df = pd.read_csv('data.csv')
df = df.drop('inhibition_value', axis=1)
df = df.rename(columns={'label_cls': 'inhibition'})
df

In [5]:

MODEL_NAME = "multimolecule/rnafm"
MAX_LENGTH = 512
BATCH_SIZE = 8
NUM_FOLDS = 5
SEED = 42

In [6]:
tokenizer = RnaTokenizer.from_pretrained(MODEL_NAME)
sep_token = tokenizer.sep_token if tokenizer.sep_token else "[SEP]"

def combine_sequences(sirna, mrna):
    sirna_str = str(sirna)
    mrna_str = str(mrna)
    return f"{sirna_str}{sep_token}{mrna_str}"

df['combined_sequence'] = df.apply(lambda row: combine_sequences(row['siRNA_sequence'], row['mRNA_sequence']), axis=1)
all_sequences = df['combined_sequence'].tolist()
all_labels = df['inhibition'].tolist()

In [7]:
class SiRnaClassificationDataset(Dataset):
    def __init__(self, sequences, targets, tokenizer, max_len):
        self.sequences = sequences
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence = str(self.sequences[idx])
        target = int(self.targets[idx])

        encoding = self.tokenizer.encode_plus(
            sequence, add_special_tokens=True, max_length=self.max_len,
            return_token_type_ids=False, padding='max_length', truncation=True,
            return_attention_mask=True, return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(target, dtype=torch.float)
        }

In [8]:
def mean_pooling(model_output, attention_mask):
    if isinstance(model_output, (tuple, list)):
        token_embeddings = model_output[0]
    else:
        token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

pooling_strategy = mean_pooling

In [9]:
def create_simple_classifier(input_size): return nn.Linear(input_size, 1)
def create_medium_mlp_classifier(input_size, hidden_dim=256):
    return nn.Sequential(
        nn.Linear(input_size, hidden_dim), nn.ReLU(), nn.Dropout(0.1),
        nn.Linear(hidden_dim, 1)
    )
def create_deep_mlp_classifier(input_size, hidden_dims=[512, 256, 128]):
    layers = []
    current_dim = input_size
    for h_dim in hidden_dims:
        layers.extend([nn.Linear(current_dim, h_dim), nn.ReLU(), nn.Dropout(0.1)])
        current_dim = h_dim
    layers.append(nn.Linear(current_dim, 1))
    return nn.Sequential(*layers)

In [11]:
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.1
LORA_TARGET_MODULES = [
    "encoder.layer.0.attention.self.query", "encoder.layer.0.attention.self.key", "encoder.layer.0.attention.self.value",
    "encoder.layer.1.attention.self.query", "encoder.layer.1.attention.self.key", "encoder.layer.1.attention.self.value",
    "encoder.layer.2.attention.self.query", "encoder.layer.2.attention.self.key", "encoder.layer.2.attention.self.value",
    "encoder.layer.3.attention.self.query", "encoder.layer.3.attention.self.key", "encoder.layer.3.attention.self.value",
    "encoder.layer.4.attention.self.query", "encoder.layer.4.attention.self.key", "encoder.layer.4.attention.self.value",
    "encoder.layer.5.attention.self.query", "encoder.layer.5.attention.self.key", "encoder.layer.5.attention.self.value",
    "encoder.layer.6.attention.self.query", "encoder.layer.6.attention.self.key", "encoder.layer.6.attention.self.value",
    "encoder.layer.7.attention.self.query", "encoder.layer.7.attention.self.key", "encoder.layer.7.attention.self.value",
    "encoder.layer.8.attention.self.query", "encoder.layer.8.attention.self.key", "encoder.layer.8.attention.self.value",
    "encoder.layer.9.attention.self.query", "encoder.layer.9.attention.self.key", "encoder.layer.9.attention.self.value",
    "encoder.layer.10.attention.self.query", "encoder.layer.10.attention.self.key", "encoder.layer.10.attention.self.value",
    "encoder.layer.11.attention.self.query", "encoder.layer.11.attention.self.key", "encoder.layer.11.attention.self.value",
]

peft_config = LoraConfig(
    task_type=TaskType.FEATURE_EXTRACTION,
    inference_mode=False,
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    target_modules=LORA_TARGET_MODULES
)

In [12]:
class RnaFmForSequenceClassificationLora(nn.Module):
    def __init__(self, model_name, peft_config, head_type='medium', pooling_func=mean_pooling):
        super().__init__()
        try:
            self.base_model = AutoModel.from_pretrained(model_name)
            print(f"Loaded base model ({model_name}) via AutoModel.")
        except Exception as e:
            print(f"AutoModel failed ({e}), trying RnaFmModel directly...")
            self.base_model = RnaFmModel.from_pretrained(model_name)
            print(f"Loaded base model ({model_name}) via RnaFmModel.")

        self.peft_model = get_peft_model(self.base_model, peft_config)
        print("PEFT model created. Trainable parameters overview:")
        self.peft_model.print_trainable_parameters()

        hidden_size = self.base_model.config.hidden_size
        self.pooling = pooling_func

        print(f"Creating classification head of type: {head_type}")
        if head_type == 'simple': self.classifier = create_simple_classifier(hidden_size)
        elif head_type == 'medium': self.classifier = create_medium_mlp_classifier(hidden_size)
        elif head_type == 'deep': self.classifier = create_deep_mlp_classifier(hidden_size)
        else: raise ValueError("Invalid head_type.")

        print("Model initialization complete.")

    def forward(self, input_ids, attention_mask):
        outputs = self.peft_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=True
        )
        pooled_output = self.pooling(outputs, attention_mask)
        logits = self.classifier(pooled_output)
        return logits.squeeze(-1)

In [13]:
def initialize_lora_model_and_optimizer(model_name, peft_config, head_type, learning_rate, weight_decay):
    model = RnaFmForSequenceClassificationLora(model_name, peft_config, head_type=head_type, pooling_func=pooling_strategy)
    model.to(device)

    optimizer = optim.AdamW(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=learning_rate,
        weight_decay=weight_decay
    )
    print("\nOptimizer initialized for trainable parameters (LoRA + Head).")

    criterion = nn.BCEWithLogitsLoss()

    return model, optimizer, criterion

In [14]:
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 0.01
EPOCHS = 10
HEAD_TYPE = 'medium'

In [15]:
def train_epoch(model, data_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0.0
    start_time = time.time()
    for i, batch in enumerate(data_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        optimizer.zero_grad()
        logits = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(data_loader)
    elapsed_time = time.time() - start_time
    return avg_loss

def evaluate_model(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    all_labels, all_predictions, all_probabilities = [], [], []
    start_time = time.time()
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            logits = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = criterion(logits, labels)
            total_loss += loss.item()
            probabilities = torch.sigmoid(logits)
            predictions = (probabilities > 0.5).int()
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())
            all_probabilities.extend(probabilities.cpu().numpy())

    avg_loss = total_loss / len(data_loader)
    all_labels, all_predictions, all_probabilities = np.array(all_labels), np.array(all_predictions), np.array(all_probabilities)
    accuracy = accuracy_score(all_labels, all_predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='binary', zero_division=0)
    try: roc_auc = roc_auc_score(all_labels, all_probabilities)
    except ValueError: roc_auc = float('nan')
    elapsed_time = time.time() - start_time
    return avg_loss, accuracy, precision, recall, f1, roc_auc, all_labels, all_predictions

def run_training_fold(model, optimizer, criterion, train_loader, val_loader, epochs, device, fold_num):
    best_val_loss = float('inf')
    best_adapter_dir = None
    best_head_path = None
    train_losses, val_losses, metrics_history = [], [], []

    fold_save_base = f"./fold_{fold_num}_best_lora"
    adapter_save_dir = os.path.join(fold_save_base, "adapter")
    head_save_path = os.path.join(fold_save_base, "head.pth")

    os.makedirs(adapter_save_dir, exist_ok=True)

    print(f"Starting training for Fold {fold_num}, {epochs} epochs...")
    total_fold_start_time = time.time()

    for epoch in range(epochs):
        epoch_start_time = time.time()
        avg_train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
        train_losses.append(avg_train_loss)
        avg_val_loss, val_acc, val_prec, val_rec, val_f1, val_auc, _, _ = evaluate_model(model, val_loader, criterion, device)
        val_losses.append(avg_val_loss)
        metrics_history.append({'epoch': epoch + 1, 'train_loss': avg_train_loss, 'val_loss': avg_val_loss,
                                'val_accuracy': val_acc, 'val_precision': val_prec, 'val_recall': val_rec,
                                'val_f1': val_f1, 'val_roc_auc': val_auc})
        epoch_time = time.time() - epoch_start_time
        print(f"Epoch {epoch+1}/{epochs} | Time: {epoch_time:.2f}s | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.4f} | Val F1: {val_f1:.4f} | Val AUC: {val_auc:.4f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            print(f"  -> New best validation loss: {best_val_loss:.4f}. Saving adapter and head...")
            model.peft_model.save_pretrained(adapter_save_dir)
            torch.save(model.classifier.state_dict(), head_save_path)
            best_adapter_dir = adapter_save_dir
            best_head_path = head_save_path

        if device == torch.device("mps"): torch.mps.empty_cache()
        gc.collect()

    total_fold_time = time.time() - total_fold_start_time
    print(f"\nTraining finished for Fold {fold_num}. Total time: {total_fold_time:.2f}s")

    return best_adapter_dir, best_head_path, train_losses, val_losses, metrics_history, best_val_loss

In [ ]:
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=SEED)
fold_results = []
all_metrics_history = []
saved_model_paths = []

full_dataset = SiRnaClassificationDataset(all_sequences, all_labels, tokenizer, MAX_LENGTH)

print(f"\n--- Starting {NUM_FOLDS}-Fold Cross-Validation with LoRA ---")
cv_start_time = time.time()

for fold, (train_idx, val_idx) in enumerate(kf.split(all_sequences)):
    fold_num = fold + 1
    print(f"\n==================== Fold {fold_num}/{NUM_FOLDS} ====================")

    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(full_dataset, batch_size=BATCH_SIZE, sampler=train_sampler, num_workers=0)
    val_loader = DataLoader(full_dataset, batch_size=BATCH_SIZE, sampler=val_sampler, num_workers=0)
    print(f"Fold {fold_num}: Train samples = {len(train_idx)}, Validation samples = {len(val_idx)}")

    model, optimizer, criterion = initialize_lora_model_and_optimizer(
        MODEL_NAME, peft_config, HEAD_TYPE, LEARNING_RATE, WEIGHT_DECAY
    )

    best_adapter_dir, best_head_path, train_losses, val_losses, metrics_history, fold_best_val_loss = run_training_fold(
        model, optimizer, criterion, train_loader, val_loader, EPOCHS, device, fold_num
    )
    all_metrics_history.append(metrics_history)
    if best_adapter_dir:
        saved_model_paths.append(os.path.dirname(best_adapter_dir))

    if best_adapter_dir and best_head_path and os.path.exists(best_adapter_dir) and os.path.exists(best_head_path):
        print("\nLoading best model state for final evaluation...")
        eval_base_model = AutoModel.from_pretrained(MODEL_NAME)
        eval_peft_model = PeftModel.from_pretrained(eval_base_model, best_adapter_dir, is_trainable=False)
        eval_model = nn.Module()
        eval_model.peft_model = eval_peft_model
        hidden_size = eval_base_model.config.hidden_size
        if HEAD_TYPE == 'simple': eval_model.classifier = create_simple_classifier(hidden_size)
        elif HEAD_TYPE == 'medium': eval_model.classifier = create_medium_mlp_classifier(hidden_size)
        else: eval_model.classifier = create_deep_mlp_classifier(hidden_size)
        eval_model.classifier.load_state_dict(torch.load(best_head_path))
        eval_model.to(device)
        eval_model.eval()

        def eval_forward(input_ids, attention_mask):
            outputs = eval_model.peft_model(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
            pooled = pooling_strategy(outputs, attention_mask)
            logits = eval_model.classifier(pooled)
            return logits.squeeze(-1)
        eval_model.forward = eval_forward

        print(f"Performing final evaluation for Fold {fold_num}...")
        final_val_loss, final_acc, final_prec, final_rec, final_f1, final_auc, fold_labels, fold_preds = evaluate_model(
            eval_model, val_loader, criterion, device
        )

        print(f"\nFold {fold_num} Final Validation Metrics (Best LoRA Model):")
        print(f"  Loss:      {final_val_loss:.4f}")
        print(f"  Accuracy:  {final_acc:.4f}")
        print(f"  Precision: {final_prec:.4f}")
        print(f"  Recall:    {final_rec:.4f}")
        print(f"  F1 Score:  {final_f1:.4f}")
        print(f"  ROC AUC:   {final_auc:.4f}")

        fold_results.append({'fold': fold_num, 'accuracy': final_acc, 'precision': final_prec,
                             'recall': final_rec, 'f1': final_f1, 'roc_auc': final_auc,
                             'best_val_loss': fold_best_val_loss})
        try:
            cm = confusion_matrix(fold_labels, fold_preds)
            disp = ConfusionMatrixDisplay(confusion_matrix=cm)
            disp.plot(cmap=plt.cm.Blues); plt.title(f'Fold {fold_num} - Confusion Matrix (Best LoRA Model)'); plt.show()
        except Exception as e: print(f"Could not display confusion matrix: {e}")

    else:
        print(f"\nSkipping final evaluation for Fold {fold_num} as best model state was not saved.")
        fold_results.append({'fold': fold_num, 'accuracy': np.nan, 'precision': np.nan,
                             'recall': np.nan, 'f1': np.nan, 'roc_auc': np.nan,
                             'best_val_loss': np.nan})


    plt.figure(figsize=(8, 4)); plt.plot(range(1, EPOCHS + 1), train_losses, label='Training Loss', marker='o'); plt.plot(range(1, EPOCHS + 1), val_losses, label='Validation Loss', marker='x'); plt.xlabel('Epoch'); plt.ylabel('Loss (BCE)'); plt.title(f'Fold {fold_num} - Training & Validation Loss (LoRA)'); plt.legend(); plt.grid(True); plt.show()


    print(f"Cleaning up Fold {fold_num} resources...")
    del model, optimizer, criterion, train_loader, val_loader, train_sampler, val_sampler
    if 'eval_model' in locals(): del eval_model
    if 'eval_base_model' in locals(): del eval_base_model
    if 'eval_peft_model' in locals(): del eval_peft_model
    if device == torch.device("mps"): torch.mps.empty_cache()
    gc.collect()

cv_end_time = time.time()
print(f"\n--- LoRA Cross-Validation Finished --- Total Time: {cv_end_time - cv_start_time:.2f}s ---")

print("\nCleaning up saved model directories...")
for path in saved_model_paths:
    if os.path.exists(path):
        try:
            shutil.rmtree(path)
            print(f"  Removed: {path}")
        except Exception as e:
            print(f"  Error removing {path}: {e}")
print("Cleanup complete.")

In [ ]:
results_df = pd.DataFrame(fold_results)

print("\n--- LoRA Cross-Validation Summary ---")
print(results_df[[
    'fold', 'accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'best_val_loss'
]].round(4).to_string(index=False))

print("\nAverage Metrics Across Folds (LoRA):")
avg_accuracy = results_df['accuracy'].mean()
std_accuracy = results_df['accuracy'].std()
avg_precision = results_df['precision'].mean()
std_precision = results_df['precision'].std()
avg_recall = results_df['recall'].mean()
std_recall = results_df['recall'].std()
avg_f1 = results_df['f1'].mean()
std_f1 = results_df['f1'].std()
avg_roc_auc = results_df['roc_auc'].mean()
std_roc_auc = results_df['roc_auc'].std()

print(f"  Accuracy:  {avg_accuracy:.4f} +/- {std_accuracy:.4f}")
print(f"  Precision: {avg_precision:.4f} +/- {std_precision:.4f}")
print(f"  Recall:    {avg_recall:.4f} +/- {std_recall:.4f}")
print(f"  F1 Score:  {avg_f1:.4f} +/- {std_f1:.4f}")
print(f"  ROC AUC:   {avg_roc_auc:.4f} +/- {std_roc_auc:.4f}")